In [1]:
#Import libraries
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
#Download data
data = pd.read_csv("../calls_final.csv")

In [3]:
#Preprocessing
data = data.drop(["Unnamed: 0","Unnamed: 0.1","tmed","tmin","tmax","velmedia","racha"],axis=1)

In [4]:
data["type0"] = data["type0"].astype(np.int32)
data["type1"] = data["type1"].astype(np.int32)
data["type2"] = data["type2"].astype(np.int32)
data["type3"] = data["type3"].astype(np.int32)
data["wd"] = data["wd"].astype("category")
data["prec"] = data["prec"].astype("int").astype("category")

In [5]:
date = data["d"].astype("str")+"-"+data["m"].astype("str")+"-"+(data["y"]+6).astype("str")
date = pd.to_datetime(date, dayfirst=True, format='%d-%m-%Y')
data = data.drop(["d","y",'m'],axis=1)

In [6]:
data = data.drop(["type0","type1","type2"],axis=1)

In [7]:
data = pd.concat([data.drop("wd",axis=1), pd.get_dummies(data['wd'],prefix="wd",drop_first=True)], axis=1)

In [8]:
col=data.columns

n = len(data)
train_0 = data[0:int(n*0.7)]
val_0 = data[int(n*0.7):int(n*0.9)]
test_0 = data[int(n*0.9):]

In [9]:
print(f'Training data: from {min(date[0:int(n*0.7)])} to {max(date[0:int(n*0.7)])}')
print(f'Validation data: from {min(date[int(n*0.7):int(n*0.9)])} to {max(date[int(n*0.7):int(n*0.9)])}')
print(f'Testing data: from {min(date[int(n*0.9):])} to {max(date[int(n*0.9):])}')

Training data: from 2007-09-01 00:00:00 to 2015-06-25 00:00:00
Validation data: from 2015-06-26 00:00:00 to 2017-09-18 00:00:00
Testing data: from 2017-09-19 00:00:00 to 2018-10-31 00:00:00


In [10]:
minx = min(train_0["type3"])
maxx = max(train_0["type3"])

train_0["type3"] = (train_0["type3"]-minx)/(maxx-minx)
val_0["type3"] = (val_0["type3"]-minx)/(maxx-minx)
test_0["type3"] = (test_0["type3"]-minx)/(maxx-minx)
data["type3"] = (data["type3"]-minx)/(maxx-minx)

<ipython-input-10-12bdc9b6a721>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_0["type3"] = (train_0["type3"]-minx)/(maxx-minx)
<ipython-input-10-12bdc9b6a721>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_0["type3"] = (val_0["type3"]-minx)/(maxx-minx)
<ipython-input-10-12bdc9b6a721>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

In [11]:
# Window Generator for everything
class WindowGenerator():
    def __init__(self, input_width, label_width, shift, train_df, 
                 val_df, test_df, full_df, label_columns=None):
        '''
        Function to initialize the creation of the windows for the neural network.
        
        Args:
            input_width (int): Number of days for each window.
            label_width (int): Days to predict with the window.
            shift (int): Days to predict needed to add to the total window size.
            train_df (pd.DataFrame): Dataframe with the training data already normalize.
            val_df (pd.DataFrame): Dataframe with the validation data already normalize.
            test_df (pd.DataFrame): Dataframe with the testing data already normalize.
            full_df (pd.DataFrame): Dataframe with the full dataset already normalize.
            label_columns (list): list with the columns of the Dataframe to be our target variable.
        '''
        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df
        self.full_df = full_df
        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in
                                        enumerate(label_columns)}
        self.column_indices = {name: i for i, name in
                               enumerate(train_df.columns)}
        # Work out the window parameters.
        self.input_width = input_width #Input of each batch
        self.label_width = label_width #Prediction of each batch
        self.shift = shift
        self.total_window_size = input_width + shift
        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]
        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]
    def __repr__(self):
        '''
        Function to print the overall information of the windows
        '''
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])
def split_window(self, features):
    '''
    Function to split the inputs and labels (targets) windows.
    '''
    inputs = features[:, self.input_slice, :]
    labels = features[:, self.labels_slice, :]
    if self.label_columns is not None:
        labels = tf.stack(
            [labels[:, :, self.column_indices[name]] for name in self.label_columns],
            axis=-1)
    # Slicing doesn't preserve static shape information, so set the shapes
    # manually. This way the `tf.data.Datasets` are easier to inspect.
    inputs.set_shape([None, self.input_width, None])
    labels.set_shape([None, self.label_width, None])
    return inputs, labels

WindowGenerator.split_window = split_window


def make_dataset(self, data):
    '''
    Function to create the dataset splitted by inputs and labels (targets) and by temporal batches.
    '''
    data = np.array(data, dtype=np.float32)
    ds = tf.keras.preprocessing.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      batch_size=16,)
    ds = ds.map(self.split_window)
    return ds

WindowGenerator.make_dataset = make_dataset

@property
def train(self):
    '''
    Function to create the temporal dataset for the training set.
    '''
    return self.make_dataset(self.train_df)

@property
def val(self):
    '''
    Function to create the temporal dataset for the validation set.
    '''
    return self.make_dataset(self.val_df)

@property
def test(self):
    '''
    Function to create the temporal dataset for the testing set.
    '''    
    return self.make_dataset(self.test_df)

@property
def full(self):
    '''
    Function to create the temporal dataset for the full set.
    '''    
    return self.make_dataset(self.full_df)

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.full = full


In [12]:
tf.random.set_seed(613)

MAX_EPOCHS = 80
INPUT_WIDTH = 7*3 # Days of input in each batch
OUT_STEPS = 7
LABEL_WIDTH = 7 # Days of predictions
num_features=8
multi_window = WindowGenerator(input_width=INPUT_WIDTH, label_width=LABEL_WIDTH, shift=OUT_STEPS,
                               train_df=train_0, val_df=val_0, test_df=test_0, full_df = data,
                              label_columns=["type3"])

In [13]:
model=tf.keras.Sequential()

model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, 
                             return_sequences=False,
#                             recurrent_dropout = 0.7,
#                             dropout = 0.7,
                                activation="relu"
                            )))

model.add(tf.keras.layers.Dense(OUT_STEPS*1,
                          kernel_initializer=tf.initializers.zeros))

model.add(tf.keras.layers.Reshape([OUT_STEPS, 1]))

In [14]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=4,
                                                    mode='min')
model.compile(loss=tf.losses.MeanAbsoluteError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

#model training
model.fit(multi_window.train, epochs=MAX_EPOCHS,
                      validation_data=multi_window.val,
         callbacks=[early_stopping])

Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
Label indices: [21 22 23 24 25 26 27]
Label column name(s): ['type3']> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
Label indices: [21 22 23 24 25 26 27]
Label column name(s): ['type3']> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Epoch 1/80
177/177 [==============================] 

In [15]:
predtrain0=model.predict(multi_window.train)

for i in range(predtrain0.shape[0]):
    predtrain0[i]=predtrain0[i]*(maxx-minx)+minx
    for j in range(predtrain0.shape[1]):
        predtrain0[i][j][0]=np.round(predtrain0[i][j][0])

for i in range(predtrain0.shape[1]):
    df1=pd.DataFrame(predtrain0[:,i,:],columns=["type3"])
    df2=pd.DataFrame(date.iloc[list(range(21,(int(n*0.7)-6)))][:]).reset_index()
    df1.join(df2).to_csv('.\output\out-train'+str(i+1)+'t3.csv')

In [16]:
predval0=model.predict(multi_window.val)

for i in range(predval0.shape[0]):
    predval0[i]=predval0[i]*(maxx-minx)+minx
    for j in range(predval0.shape[1]):
        predval0[i][j][0]=np.round(predval0[i][j][0])

for i in range(predval0.shape[1]):
    df1=pd.DataFrame(predval0[:,i,:],columns=["type3"])
    df2=pd.DataFrame(date.iloc[list(range(21,(int(n*0.7)-6)))][:]).reset_index()
    df1.join(df2).to_csv('.\output\out-val'+str(i+1)+'t3.csv')

In [17]:
predtest0=model.predict(multi_window.test)

for i in range(predtest0.shape[0]):
    predtest0[i]=predtest0[i]*(maxx-minx)+minx
    for j in range(predtest0.shape[1]):
        predtest0[i][j][0]=np.round(predtest0[i][j][0])

for i in range(predtest0.shape[1]):
    df1=pd.DataFrame(predtest0[:,i,:],columns=["type3"])
    df2=pd.DataFrame(date.iloc[list(range(21+int(n*0.9),(len(data)-6)))][:]).reset_index()
    df1.join(df2).to_csv('.\output\out-test'+str(i+1)+'t3.csv')

In [18]:
predfull0=model.predict(multi_window.full)

for i in range(predfull0.shape[0]):
    predfull0[i]=predfull0[i]*(maxx-minx)+minx
    for j in range(predfull0.shape[1]):
        predfull0[i][j][0]=np.round(predfull0[i][j][0])

for i in range(predfull0.shape[1]):
    df1=pd.DataFrame(predfull0[:,i,:],columns=["type3"])
    df2=pd.DataFrame(date.iloc[list(range(21,(len(data)-6)))][:]).reset_index()
    df1.join(df2).to_csv('.\output\out-full'+str(i+1)+'t3.csv')